## 简单例子

In [1]:
from illufly.chat import ChatQwen, ChatZhipu

print(ChatZhipu.__doc__)


    对话智能体是基于大模型实现的智能体，可以用于对话生成、对话理解等场景。
    

ChatZhipu 参数列表：
- model: 模型名称
- end_chk: 是否在最后输出一个 EndBlock
- fetching_context: 上下文提取标记，可通过修改环境变量 ILLUFLY_CONTEXT_START 和 ILLUFLY_CONTEXT_END 修改默认值
- fetching_final_answer: 最终答案提取标记，可通过修改环境变量 ILLUFLY_FINAL_ANSWER_START 和 ILLUFLY_FINAL_ANSWER_END 修改默认值
- fetching_output: 输出内容提取标记
- chat_learn_folder: 知识库目录，可通过修改环境变量 ILLUFLY_CHAT_LEARN 修改默认值
- default_docs: 默认文档目录，可通过修改环境变量 ILLUFLY_DOCS 修改默认值
- func: 用于自定义工具的同步执行函数
- async_func: 用于自定义工具的异步执行函数
- name: 工具名称
- handlers: EventBlock 迭代器处理函数列表，默认为 [log]，当调用 call 方法时，会使用该列表中的函数逐个处理 EventBlock
- block_processor: 在 yield 之前将 EventBlock 事件转换为新的格式，在 __call__ 方法的输出生成器时使用
- threads_group: 如果由 illufly 管理线程池实现并发或异步，则可以指定线程组名称，默认为 DEFAULT
- providers: 实例的 consumer_dict 属性由 providers 列表中每个 Runnable 的 provider_dict 属性提供
- consumers: 实例的 provider_dict 属性将被 consumers 列表中每个 Runnable 引用
- dynamic_providers: 如果实例在不同周期中重复使用，可能会希望先在绑定前先清除旧的绑定，此时就应该使用动态绑定，即执行 bind_provider 时提供 dynamic=True 参数
- lazy_binding_map: 有时你

In [2]:
from illufly.chat import ChatQwen

def poet(input: str):
    """我是诗人，擅长作诗。"""
    return "请看我的新作：\n大海啊, 全是水"

chat = ChatQwen(tools=[poet])
chat("帮我写一首小诗？")

[AGENT] ChatQwen.4371148192


[FINAL_TOOLS_CALL] [{"index": 0, "id": "call_4758dc72ef8e4172aa5d55", "type": "function", "function": {"name": "poet", "arguments": "{\"input\": \"帮我写一首小诗\"}"}}]
[AGENT] poet
请看我的新作：
大海啊, 全是水
这是我为你创作的小诗：
大海啊, 全是水
希望你会喜欢。当然，如果你有更具体的要求，比如风格、主题等，也可以告诉我，我可以尝试创作出更符合你期望的作品。


'这是我为你创作的小诗：\n大海啊, 全是水\n希望你会喜欢。当然，如果你有更具体的要求，比如风格、主题等，也可以告诉我，我可以尝试创作出更符合你期望的作品。'

In [2]:
chat.final_answer

''

## 综合例子

In [1]:
# 创建一个工具
from illufly.chat import ChatQwen
from illufly.types import BaseAgent, PromptTemplate, EventBlock

def get_current_weather(location: str):
    """获取城市的天气情况"""
    # return f"{location}今天是晴天。 "
    yield self.create_event_block("chunk", f"{location}今天是晴天。 ")

## 声明大模型
qwen = ChatQwen(
    name="qwen",
    tools=[BaseAgent(get_current_weather)],
    knowledge=["这里的兔子是指嫦娥的宠物兔子"],
    memory=[("system", "你是一个作家")]
)

qwen("先查询广州天气，然后结合兔子的主人，帮我写一首关于兔子的四句打油诗", verbose=False)

[AGENT] qwen


[FINAL_TOOLS_CALL] [{"index": 0, "id": "call_4105673da2924b77a8bd58", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}]
[AGENT] get_current_weather
广州今天是晴天。 
好的，已知广州今天是晴天，而兔子的主人是嫦娥。接下来，我将为你创作一首关于兔子的四句打油诗：

玉兔晒暖在月边，
广寒宫冷盼晴天。
嫦娥抚袖笑颜开，
千里遥祝广州安。


'好的，已知广州今天是晴天，而兔子的主人是嫦娥。接下来，我将为你创作一首关于兔子的四句打油诗：\n\n玉兔晒暖在月边，\n广寒宫冷盼晴天。\n嫦娥抚袖笑颜开，\n千里遥祝广州安。'

In [2]:
qwen.provider_dict

{'last_output': '好的，已经查到广州今天的天气是晴天。接下来，我将为您创作一首关于兔子的四句打油诗。\n\n阳光明媚在广州，小兔蹦跳乐逍遥。\n主人宠爱如珍宝，绒球摇曳幸福高。',
 'resources': [],
 'task': '先查询广州天气，然后结合兔子的主人，帮我写一首关于兔子的四句打油诗',
 'final_answer': '',
 'tools_calling_steps': [],
 'tools_name': 'get_current_weather',
 'tools_desc': '{"type": "function", "function": {"name": "get_current_weather", "description": "获取城市的天气情况", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": ""}}, "required": ["location"]}}}',
 'knowledge': ['这里的兔子是指嫦娥的宠物兔子']}

In [2]:
qwen.memory

[{'role': 'system', 'content': '你是一个作家'},
 {'role': 'user', 'content': '已知知识：\n这里的兔子是指嫦娥的宠物兔子'},
 {'role': 'assistant', 'content': 'OK'},
 {'role': 'user', 'content': '先查询广州天气，然后结合兔子的主人，帮我写一首关于兔子的四句打油诗'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_13c5bdfd6bd5410faebd1d',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'tool_call_id': 'call_13c5bdfd6bd5410faebd1d',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '广州今天是晴天。 '},
 {'role': 'assistant',
  'content': '在广州的晴朗一天里，\n小白兔，你欢快地蹦跳，\n嫦娥姐姐在天空微笑着看你，\n阳光下，你的身影如此生动。'}]

## 多轮对话

使用`messages`管理多轮对话的记忆。<br>
你也可以在`chat`函数中通过参数`k`限制希望保留的对话轮次，但这不影响`messages`记录完整的对话过程。

In [1]:
from illufly.chat import FakeLLM, ChatZhipu, ChatQwen
from illufly.io import log, alog

a = ChatQwen()

In [2]:
a.build_chat_memory("hi")

[{'role': 'user', 'content': 'hi'}]

## 回调工具

In [1]:
from illufly.chat import ChatQwen
from illufly.types import BaseAgent

def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

qwen = ChatQwen(tools=[BaseAgent(get_current_weather)])
qwen("今天广州天气如何啊")



[TOOLS_CALL_FINAL] [{"index": 0, "id": "call_b1beefa5dde64281b26226", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}]
广州今天是晴天。 
广州今天的天气是晴天。


'广州今天的天气是晴天。'

In [8]:
qwen.tools[0].tool_desc

{'type': 'function',
 'function': {'name': 'get_current_weather',
  'description': '获取城市的天气情况',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string', 'description': ''}},
   'required': ['location']}}}

In [9]:
qwen("这是怎么得出来的？")

我是通过使用天气获取功能查询得到的。我查询了广州今天的天气，反馈的结果是晴天。这就是依据。如果你想查看当时的查询结果，你可以让我再做一次查询。

广州今天是晴天。 
这是通过查询当前天气接口得出的，广州今天的天气是晴天。


'这是通过查询当前天气接口得出的，广州今天的天气是晴天。'

In [10]:
qwen("再看看上海")



上海今天是晴天。 
好的，通过查询当前天气的接口得知，上海今天的天气也是晴天。


'好的，通过查询当前天气的接口得知，上海今天的天气也是晴天。'

## 识图模型

### 识别在线图片

In [1]:
from illufly.chat import ChatQwenVL
from illufly.types import Messages

qwen = ChatQwenVL()
prompt = [
    (
        "user", 
        [
            {'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'},
            {'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'},
            {'text': '这些是什么'}
        ]
    )
]
qwen(prompt)

图1中是一名女子和她的宠物狗在沙滩上互动，狗狗抬起前爪似乎想要握手。
图2是一张老虎的CG渲染作品。


'图1中是一名女子和她的宠物狗在沙滩上互动，狗狗抬起前爪似乎想要握手。\n图2是一张老虎的CG渲染作品。'

In [19]:
prompt

[('user',
  [{'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'},
   {'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'},
   {'text': '这些是什么'}])]

In [20]:
qwen.build_chat_memory(prompt)

[{'role': 'user',
  'content': [{'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'},
   {'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'},
   {'text': '这些是什么'}]},
 {'role': 'user',
  'content': [{'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'},
   {'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'},
   {'text': '这些是什么'}]},
 {'role': 'user',
  'content': [{'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'},
   {'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'},
   {'text': '这些是什么'}]}]

In [2]:
qwen.memory

[{'role': 'user',
  'content': [{'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'},
   {'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'},
   {'text': '这些是什么'}]},
 {'role': 'assistant',
  'content': [{'text': '图1中是一名女子和她的宠物狗在沙滩上互动，狗狗抬起前爪似乎想要握手。\n图2是一张老虎的CG渲染作品。'}]}]

In [3]:
qwen.task

[{'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'},
 {'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'},
 {'text': '这些是什么'}]

### 识别本地图片

In [1]:
from illufly.chat import ChatQwenVL
from illufly.types import Messages

qwen = ChatQwenVL()

qwen(('user', [{'image': 'images/girl.jpg'}, {'text': '图片中是什么?'}]))

[USER] {"image": "images/girl.jpg"},图片中是什么?
这张图是一幅描绘古代女子的插画。她身穿绿色汉服，坐在池塘边的一块木板上。她的头发盘起，并用发饰装饰着，在阳光下显得格外美丽。

这位女士的表情平静而安详，似乎在享受这宁静的时刻。周围的环境充满了自然的气息——树木、水生植物和盛开的花朵都为画面增添了生动感。整体来说，这是一张充满诗意与古风美的艺术作品。


'这张图是一幅描绘古代女子的插画。她身穿绿色汉服，坐在池塘边的一块木板上。她的头发盘起，并用发饰装饰着，在阳光下显得格外美丽。\n\n这位女士的表情平静而安详，似乎在享受这宁静的时刻。周围的环境充满了自然的气息——树木、水生植物和盛开的花朵都为画面增添了生动感。整体来说，这是一张充满诗意与古风美的艺术作品。'